In [ ]:
import re
import pandas as pd
pd.options.io.excel.xlsx.writer = 'xlsxwriter'


def extrac_year(name):
    publish = int(name.split('-')[1])
    title = name.split('-')[0]
    try:
        year = re.search(r'20\d{2}', title).group(0)
    except:
        year = publish-1
    return year


path = r"D:\Projects\DigitalTransformation\01.预备文件与处理\datas\年报链接(未处理).txt"
dataset_ = pd.read_csv(path,index_col=None,header=None,names=['股票代码','截止时间','时间戳','股票简称','素材id','年报标题','下载链接'],dtype={'股票代码':'object'})
dataset_.drop_duplicates(subset=['下载链接', '素材id', '股票代码'],inplace=True)

dataset_['公布年份'] = dataset_['下载链接'].apply(lambda x:x.split('/')[4][:4])
dataset_['年报标题2'] = dataset_['年报标题'] + '-' + dataset_['公布年份']
dataset_['年报年份'] = dataset_['年报标题2'].apply(lambda x: extrac_year(x))
dataset_['时间戳'] = dataset_['时间戳'].apply(lambda x:int(x)/1e3)
dataset_.sort_values(['股票代码','年报年份'],inplace=True)
dataset_.reset_index(level=0, inplace=True)
dataset_.drop(['index', '年报标题2','公布年份','素材id'], axis=1, inplace=True)
dataset_['截止时间'] = dataset_['截止时间'].astype('int')
dataset_['年报年份'] = dataset_['年报年份'].astype('int')
dataset_ = dataset_[['股票代码', '股票简称', '年报年份', '年报标题', '时间戳', '下载链接']]
dataset_

In [ ]:
# 拿到所有链接，初步筛选去除标题名称不符合要求的所有数据
delitems = dataset_[(~dataset_['年报标题'].str.contains('年度报'))&(~dataset_['年报标题'].str.contains('年报'))][['年报标题']].index.tolist()
dataset = dataset_.drop(list(delitems))
dataset

In [ ]:
# 删除各年内地修订数据
secids=set(dataset['股票代码'].tolist())
result=[]
def delfun(items):
    result_=[]
    while len(items) >1:
        res_ = {item.split(':')[1]:item.split(':')[0] for item in items}
        name_ = min(res_, key=res_.get)
        result_.append(int(name_))
        items.remove(f"{res_[name_]}:{name_}")
    return result_

for secid in secids:
    _dataset = dataset[dataset['股票代码']==secid]
    years = set(_dataset['年报年份'].tolist())
    for year in years:
        res=[]
        for idx,row in _dataset.iterrows():
            if year == row[2]:
                content=f"{int(row[4])}:{row.name}"
                res.append(content)
        if len(res)>1:
            result.extend(delfun(res))
dataset.drop(list(result),inplace=True)  # sorted(delitems)
dataset

In [ ]:
# 再次去除重复
dataset_.drop_duplicates(subset=['股票代码','年报年份'])
# 组装为目标格式
dataset['年报年份'] = dataset['年报年份'].astype('str')
dataset['title'] = dataset['股票代码'] + '-' + dataset['年报年份'] + '-' + dataset['股票简称'] + '-' +'年度报告.pdf'
result = dataset[['title','下载链接']]


In [ ]:
result.to_csv(r'D:\Projects\DigitalTransformation\01.预备文件与处理\datas\下载链接(清洗后).txt', header=None, index=None)